# Visualizing Body Data
AMASS uses [MoSh++](https://amass.is.tue.mpg.de) pipeline to fit [SMPL+H body model](https://mano.is.tue.mpg.de/)
to a human motion capture (mocap) session.
[These mocaps](https://amass.is.tue.mpg.de/dataset) are from different publicly available datasets.
A single data file in amass has the parameters to control gender, pose, shape, global translation and soft tissue dynamics
in correspondence with the original motion capture sequence. Here we present code snippets to create a body image
with these parameters. Since a mocap is a time sequence you visualize the "moshed" per frame results.

In [1]:
# Preparing the environment
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline
from ipywidgets import interact, interact_manual
from ipywidgets import IntSlider

import sys, os
import torch
import numpy as np

from human_body_prior.tools.omni_tools import copy2cpu as c2c
from os import path as osp

support_datadir = '../support_data/'

In [2]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

We assume you have downloaded the required body model and put that in body_models directory of this repository.
For SMPL+H body model, download it from http://mano.is.tue.mpg.de/.
You can obtain dynamic shape blendshapes, e.g. DMPLs, from http://smpl.is.tue.mpg.de/downloads.
If you use any of these models in your research please follow their respective citation rules.
One thing to note is that you can obtain basic SMPL+H model from their [website](http://mano.is.tue.mpg.de/),
however, this model doesn't have DMPL parameters and has only 10 betas. Doing so will reduce the accuracy of bodies provided by AMASS.

In [3]:
npz_bdata_path = osp.join(support_datadir, 'github_data/amass_sample.npz') # the path to body data
bdata = np.load(npz_bdata_path)

# you can set the gender manually and if it differs from data's then contact or interpenetration issues might happen
subject_gender = bdata['gender']

print('Data keys available:%s'%list(bdata.keys()))

print('The subject of the mocap sequence is  {}.'.format(subject_gender))

Data keys available:['poses', 'gender', 'mocap_framerate', 'betas', 'marker_data', 'dmpls', 'marker_labels', 'trans']
The subject of the mocap sequence is  male.


In [4]:
from human_body_prior.body_model.body_model import BodyModel

bm_path = osp.join(support_datadir, 'body_models/smplh/{}/model.npz'.format(subject_gender))
dmpl_path = osp.join(support_datadir, 'body_models/dmpls/{}/model.npz'.format(subject_gender))

num_betas = 16 # number of body parameters
num_dmpls = 8 # number of DMPL parameters

bm = BodyModel(bm_path=bm_path, num_betas=num_betas, num_dmpls=num_dmpls, path_dmpl=dmpl_path).to(comp_device)
faces = c2c(bm.f)


The provided sample data also has the original mocap marker data.
In the real AMASS dataset, only markers for the test set are included.
For the rest of the subsets you can obtain the marker data from their respective websites.
In the following we make PyTorch tensors for parameters controlling different part of the body model.

**Please note how pose indices for different body parts work.**

In [5]:
time_length = len(bdata['trans'])

body_parms = {
    'root_orient': torch.Tensor(bdata['poses'][:, :3]).to(comp_device), # controls the global root orientation
    'pose_body': torch.Tensor(bdata['poses'][:, 3:66]).to(comp_device), # controls the body
    'pose_hand': torch.Tensor(bdata['poses'][:, 66:]).to(comp_device), # controls the finger articulation
    'trans': torch.Tensor(bdata['trans']).to(comp_device), # controls the global body position
    'betas': torch.Tensor(np.repeat(bdata['betas'][:num_betas][np.newaxis], repeats=time_length, axis=0)).to(comp_device), # controls the body shape. Body shape is static
    'dmpls': torch.Tensor(bdata['dmpls'][:, :num_dmpls]).to(comp_device) # controls soft tissue dynamics
}

print('Body parameter vector shapes: \n{}'.format(' \n'.join(['{}: {}'.format(k,v.shape) for k,v in body_parms.items()])))
print('time_length = {}'.format(time_length))

Body parameter vector shapes: 
root_orient: torch.Size([601, 3]) 
pose_body: torch.Size([601, 63]) 
pose_hand: torch.Size([601, 90]) 
trans: torch.Size([601, 3]) 
betas: torch.Size([601, 16]) 
dmpls: torch.Size([601, 8])
time_length = 601


Import the required files for viewing out mesh:

In [6]:
import trimesh
from body_visualizer.tools.vis_tools import colors
from body_visualizer.mesh.mesh_viewer import MeshViewer
from body_visualizer.mesh.sphere import points_to_spheres
from amass.tools.notebook_tools import show_image

imw, imh=1600, 1600
mv = MeshViewer(width=imw, height=imh, use_offscreen=True)

### Visualize betas and pose_body
Let's see how our body looks like using the pose and body shape parameters.
We first produce the body surface in batched mode.

Now we can visualize each frame of data

In [7]:
body_pose_beta = bm(**{k:v for k,v in body_parms.items() if k in ['pose_body', 'betas']})

def vis_body_pose_beta(fId = 0):
    body_mesh = trimesh.Trimesh(vertices=c2c(body_pose_beta.v[fId]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))
    mv.set_static_meshes([body_mesh])
    body_image = mv.render(render_wireframe=False)
    show_image(body_image)

interact_manual(vis_body_pose_beta,fId=IntSlider(min=0, max=time_length, step=1, ontinuous_update=False))

interactive(children=(IntSlider(value=0, description='fId', max=601), Button(description='Run Interact', style…

<function __main__.vis_body_pose_beta(fId=0)>

We see a male subject sitting and havig the hands open. 
Let's articulate the fingers as well:

### Visualize pose hands
To articulate fingers we use the 66:156 pose vector elements.

In [8]:
body_pose_hand = bm(**{k:v for k,v in body_parms.items() if k in ['pose_body', 'betas', 'pose_hand']})

def vis_body_pose_hand(fId = 0):
    body_mesh = trimesh.Trimesh(vertices=c2c(body_pose_hand.v[fId]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))
    mv.set_static_meshes([body_mesh])
    body_image = mv.render(render_wireframe=False)
    show_image(body_image)

interact_manual(vis_body_pose_hand,fId=IntSlider(min=0, max=time_length, step=1, ontinuous_update=False))


interactive(children=(IntSlider(value=0, description='fId', max=601), Button(description='Run Interact', style…

<function __main__.vis_body_pose_hand(fId=0)>

It seems that the subject is holding something with one hand.

### Visualize body joints

To access joint locations of the body one can use **Jtr** attribute of the returned body.
These can be visualized as spheres.
Here we render the body transparently to visualize the joints.

In [9]:
def vis_body_joints(fId = 0):
    joints = c2c(body_pose_hand.Jtr[fId])
    joints_mesh = points_to_spheres(joints, vc = colors['red'], radius=0.005)

    mv.set_static_meshes([joints_mesh])
    body_image = mv.render(render_wireframe=False)
    show_image(body_image)

interact_manual(vis_body_joints,fId=IntSlider(min=0, max=time_length, step=1, ontinuous_update=False))


interactive(children=(IntSlider(value=0, description='fId', max=601), Button(description='Run Interact', style…

<function __main__.vis_body_joints(fId=0)>

### Visualize DMPLs

You can control the soft tissue dynamics with dmpl parameters. Please have in mind, by nature of dmpl parameters being dynamic, the better appear when animation the whole sequence. Refer to full renders of the parameter sequences in our [website](https://amass.is.tue.mpg.de/).

In [10]:
body_dmpls = bm(**{k:v for k,v in body_parms.items() if k in ['pose_body', 'betas', 'pose_hand', 'dmpls']})

def vis_body_dmpls(fId = 0):
    body_mesh = trimesh.Trimesh(vertices=c2c(body_dmpls.v[fId]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))
    mv.set_static_meshes([body_mesh])
    body_image = mv.render(render_wireframe=False)
    show_image(body_image)

interact_manual(vis_body_dmpls,fId=IntSlider(min=0, max=time_length, step=1, ontinuous_update=False))


interactive(children=(IntSlider(value=0, description='fId', max=601), Button(description='Run Interact', style…

<function __main__.vis_body_dmpls(fId=0)>

### Visualizing the global root orientation

In the above examples we don't use the global translation or rotation.
However, we can globally control the character position and orientation with **trans**, and **root_orient** parameters respectively.

In [11]:
body_trans_root = bm(**{k:v for k,v in body_parms.items() if k in ['pose_body', 'betas', 'pose_hand', 'dmpls',
                                                                   'trans', 'root_orient']})

def vis_body_trans_root(fId = 0):
    body_mesh = trimesh.Trimesh(vertices=c2c(body_trans_root.v[fId]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))
    mv.set_static_meshes([body_mesh])
    body_image = mv.render(render_wireframe=False)
    show_image(body_image)

interact_manual(vis_body_trans_root,fId=IntSlider(min=0, max=time_length, step=1, ontinuous_update=False))


interactive(children=(IntSlider(value=0, description='fId', max=601), Button(description='Run Interact', style…

<function __main__.vis_body_trans_root(fId=0)>

The global root orientation of amass is so that if you render with MeshViewer you will always get a top view of the body.
One can rotate the body into front view by transforming the mesh.

In [12]:
def vis_body_transformed(fId = 0):
    body_mesh = trimesh.Trimesh(vertices=c2c(body_trans_root.v[fId]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))
    body_mesh.apply_transform(trimesh.transformations.rotation_matrix(-90, (0, 0, 1)))
    body_mesh.apply_transform(trimesh.transformations.rotation_matrix(30, (1, 0, 0)))

    mv.set_static_meshes([body_mesh])
    body_image = mv.render(render_wireframe=False)
    show_image(body_image)

interact_manual(vis_body_transformed,fId=IntSlider(min=0, max=time_length, step=1, ontinuous_update=False))

interactive(children=(IntSlider(value=0, description='fId', max=601), Button(description='Run Interact', style…

<function __main__.vis_body_transformed(fId=0)>